In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# 检查是否可以使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练的ResNet50模型
model = models.resnet50(pretrained=True)

# 修改最后的全连接层以匹配你的分类任务
num_classes = 3  # 假设我们有10个类别
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 将模型移动到GPU（如果可用）
model = model.to(device)

D:\Anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Lenovo/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:14<00:00, 6.95MB/s]


In [2]:
# 数据预处理
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 加载数据集
data_dir = './'
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}", transform=data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes


In [3]:
criterion = nn.CrossEntropyLoss()

# 只微调最后的全连接层参数，其余层参数保持冻结
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [4]:
def train_model(model, criterion, optimizer, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # 每个epoch有训练和验证两个阶段
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # 训练模式
            else:
                model.eval()   # 验证模式

            running_loss = 0.0
            running_corrects = 0

            # 遍历数据
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 梯度清零
                optimizer.zero_grad()

                # 前向传播
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 反向传播+优化(只在训练阶段)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 统计损失和准确率
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 保存最佳模型
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    # 加载最佳模型权重
    model.load_state_dict(best_model_wts)
    return model

# 训练模型
model = train_model(model, criterion, optimizer, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.0306 Acc: 0.4781
test Loss: 0.9229 Acc: 0.5909

Epoch 1/24
----------
train Loss: 0.8085 Acc: 0.6414
test Loss: 0.7727 Acc: 0.6364

Epoch 2/24
----------
train Loss: 0.7298 Acc: 0.6813
test Loss: 0.7104 Acc: 0.6818

Epoch 3/24
----------
train Loss: 0.6287 Acc: 0.7410
test Loss: 0.6014 Acc: 0.7424

Epoch 4/24
----------
train Loss: 0.5554 Acc: 0.7968
test Loss: 0.5009 Acc: 0.8182

Epoch 5/24
----------
train Loss: 0.4847 Acc: 0.8367
test Loss: 0.4073 Acc: 0.9091

Epoch 6/24
----------
train Loss: 0.4925 Acc: 0.8287
test Loss: 0.3873 Acc: 0.8636

Epoch 7/24
----------
train Loss: 0.5369 Acc: 0.7888
test Loss: 0.3504 Acc: 0.9394

Epoch 8/24
----------
train Loss: 0.4723 Acc: 0.8446
test Loss: 0.3444 Acc: 0.8788

Epoch 9/24
----------
train Loss: 0.4147 Acc: 0.8685
test Loss: 0.3345 Acc: 0.8788

Epoch 10/24
----------
train Loss: 0.4554 Acc: 0.8287
test Loss: 0.3096 Acc: 0.9394

Epoch 11/24
----------
train Loss: 0.3755 Acc: 0.8765
test Loss: 0.3081 Acc

In [7]:
def evaluate_model(model, dataloaders):
    model.eval()  # 设置为评估模式
    running_corrects = 0

    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    acc = running_corrects.double() / dataset_sizes['test']
    print(f'Validation Accuracy: {acc:.4f}')

# 评估模型
evaluate_model(model, dataloaders)

Validation Accuracy: 0.9545
